# Games reviews on metacritic from 1995 to 2021

### Imports

In [2]:
import pandas as pd
from typing import List

### Load data from CSV

In [3]:
games_reviews = pd.read_csv("all_games.csv")
games_reviews.head()

,name,platform,release_date,summary,meta_score,user_review
0,The Legend of Zelda: Ocarina of Time,Nintendo 64,"November 23, 1998","As a young boy, Link is tricked by Ganondorf, ...",99,9.1
1,Tony Hawk's Pro Skater 2,PlayStation,"September 20, 2000",As most major publishers' development efforts ...,98,7.4
2,Grand Theft Auto IV,PlayStation 3,"April 29, 2008",[Metacritic's 2008 PS3 Game of the Year; Also ...,98,7.7
3,SoulCalibur,Dreamcast,"September 8, 1999","This is a tale of souls and swords, transcendi...",98,8.4
4,Grand Theft Auto IV,Xbox 360,"April 29, 2008",[Metacritic's 2008 Xbox 360 Game of the Year; ...,98,7.9


## Data Cleanup

### Cleanup release date

In [4]:
games_reviews["release_date"] = pd.to_datetime(games_reviews["release_date"])
games_reviews["release_date"]

0       1998-11-23
1       2000-09-20
2       2008-04-29
3       1999-09-08
4       2008-04-29
           ...    
18795   2013-05-21
18796   2003-11-03
18797   2015-09-22
18798   2012-10-15
18799   2009-03-31
Name: release_date, Length: 18800, dtype: datetime64[ns]

### Use same scale from meta score in the user reviews (0-100)

In [5]:
games_reviews["meta_score"] = games_reviews["meta_score"] / 10
games_reviews["meta_score"]

0        9.9
1        9.8
2        9.8
3        9.8
4        9.8
        ... 
18795    2.2
18796    2.2
18797    2.1
18798    2.0
18799    2.0
Name: meta_score, Length: 18800, dtype: float64

### Replace "tbd" user_reviews with 0
### **WARNING!** remember to not compare user scores and meta scores when the user score is 0

In [6]:
games_reviews["user_review"] = games_reviews["user_review"].replace("tbd", 0)


### Cast user_review to float64

In [7]:
games_reviews["user_review"] = games_reviews["user_review"].astype("float64")

### Rename user_review to user_score

In [8]:
games_reviews = games_reviews.rename(columns={"user_review": "user_score"})

### Check for NAN

In [9]:
games_reviews[games_reviews.isna().any(axis=1)]

,name,platform,release_date,summary,meta_score,user_score
833,Injustice 2: Legendary Edition,PlayStation 4,2018-03-27,NaN,8.8,7.6
1472,Moto Racer Advance,Game Boy Advance,2002-12-05,NaN,8.6,0.0
1535,Mini Metro,Switch,2018-08-30,NaN,8.6,8.4
1649,Out of the Park Baseball 13,PC,2012-04-09,NaN,8.5,7.8
2427,Guild Wars 2: Path of Fire,PC,2017-09-22,NaN,8.3,7.3
...,...,...,...,...,...,...
17999,One Piece: Unlimited Cruise SP,PlayStation 4,2012-02-10,NaN,4.6,7.0
18134,AeternoBlade II,Wii U,2019-10-11,NaN,4.4,4.4
18253,Surgeon Simulator: Experience Reality,PlayStation 4,2016-12-03,NaN,4.3,4.3
18532,The Ripping Friends,Game Boy Advance,2002-03-24,NaN,3.8,0.0


#### The only field that seems to have NaN's is the summary field

### Replace NaN summary with empty string

In [10]:
games_reviews["summary"] = games_reviews["summary"].fillna("")

### Check for NaN again

In [11]:
games_reviews[games_reviews.isna().any(axis=1)]

,name,platform,release_date,summary,meta_score,user_score


## Add supporting fields to data

### Add release year, month and day

In [12]:
games_reviews["release_year"] = games_reviews["release_date"].dt.year
games_reviews["release_month"] =  games_reviews["release_date"].dt.month
games_reviews["release_month_name"] =  games_reviews["release_date"].dt.month_name()
games_reviews["release_day"] = games_reviews["release_date"].dt.day
games_reviews.head()

,name,platform,release_date,summary,meta_score,user_score,release_year,release_month,release_month_name,release_day
0,The Legend of Zelda: Ocarina of Time,Nintendo 64,1998-11-23,"As a young boy, Link is tricked by Ganondorf, ...",9.9,9.1,1998,11,November,23
1,Tony Hawk's Pro Skater 2,PlayStation,2000-09-20,As most major publishers' development efforts ...,9.8,7.4,2000,9,September,20
2,Grand Theft Auto IV,PlayStation 3,2008-04-29,[Metacritic's 2008 PS3 Game of the Year; Also ...,9.8,7.7,2008,4,April,29
3,SoulCalibur,Dreamcast,1999-09-08,"This is a tale of souls and swords, transcendi...",9.8,8.4,1999,9,September,8
4,Grand Theft Auto IV,Xbox 360,2008-04-29,[Metacritic's 2008 Xbox 360 Game of the Year; ...,9.8,7.9,2008,4,April,29


### Add mean score (meta_score, user_review)

In [13]:
def mean_score(meta_score: float, user_score: float) -> float:
    if user_score == 0:
        result = meta_score

    else:
        result = (meta_score + user_score) / 2

    return round(result, 1)


games_reviews["mean_score"] = games_reviews.apply(
    lambda x: mean_score(x["meta_score"], x["user_score"]), axis=1
)
games_reviews.head()


,name,platform,release_date,summary,meta_score,user_score,release_year,release_month,release_month_name,release_day,mean_score
0,The Legend of Zelda: Ocarina of Time,Nintendo 64,1998-11-23,"As a young boy, Link is tricked by Ganondorf, ...",9.9,9.1,1998,11,November,23,9.5
1,Tony Hawk's Pro Skater 2,PlayStation,2000-09-20,As most major publishers' development efforts ...,9.8,7.4,2000,9,September,20,8.6
2,Grand Theft Auto IV,PlayStation 3,2008-04-29,[Metacritic's 2008 PS3 Game of the Year; Also ...,9.8,7.7,2008,4,April,29,8.8
3,SoulCalibur,Dreamcast,1999-09-08,"This is a tale of souls and swords, transcendi...",9.8,8.4,1999,9,September,8,9.1
4,Grand Theft Auto IV,Xbox 360,2008-04-29,[Metacritic's 2008 Xbox 360 Game of the Year; ...,9.8,7.9,2008,4,April,29,8.9


## Top 10 rated games by meta score, user score and mean score
### *The tiebreaker for games with the same score will be the release date*

### Helper functions

In [14]:
def view_group_n(
    df: pd.DataFrame,
    groupby_fields: List[str],
    value_fields: List[str],
    agg_funcs: List[str],
    sort_by: List[str],
    asc: List[bool],
    n: int = 10,
):
    view = (
        df.groupby(groupby_fields)[value_fields]
        .agg(dict(zip(value_fields, agg_funcs)))
        .sort_values(ascending=asc, by=sort_by)[:n]
        .reset_index()
    )
    view.index += 1
    return view


## By meta score (released first)

In [15]:
view_group_n(
    games_reviews,
    groupby_fields=["name"],
    value_fields=["meta_score", "release_date"],
    agg_funcs=["max", "max"],
    sort_by=["meta_score", "release_date"],
    asc=[False, True],
)


,name,meta_score,release_date
1,The Legend of Zelda: Ocarina of Time,9.9,1998-11-23
2,Tony Hawk's Pro Skater 2,9.8,2001-08-21
3,SoulCalibur,9.8,2008-07-02
4,Grand Theft Auto IV,9.8,2008-12-02
5,NFL 2K1,9.7,2000-09-07
6,Tony Hawk's Pro Skater 3,9.7,2002-03-28
7,Grand Theft Auto III,9.7,2002-05-20
8,Metroid Prime,9.7,2002-11-17
9,Halo: Combat Evolved,9.7,2003-09-30
10,Super Mario Galaxy,9.7,2007-11-12


## By meta score (released last)

In [16]:
view_group_n(
    games_reviews,
    groupby_fields=["name"],
    value_fields=["meta_score", "release_date"],
    agg_funcs=["max", "max"],
    sort_by=["meta_score", "release_date"],
    asc=[False, False],
)


,name,meta_score,release_date
1,The Legend of Zelda: Ocarina of Time,9.9,1998-11-23
2,Grand Theft Auto IV,9.8,2008-12-02
3,SoulCalibur,9.8,2008-07-02
4,Tony Hawk's Pro Skater 2,9.8,2001-08-21
5,Disco Elysium: The Final Cut,9.7,2021-10-12
6,The House in Fata Morgana - Dreams of the Reve...,9.7,2021-04-09
7,Red Dead Redemption 2,9.7,2019-11-05
8,Super Mario Odyssey,9.7,2017-10-27
9,The Legend of Zelda: Breath of the Wild,9.7,2017-03-03
10,Grand Theft Auto V,9.7,2015-04-13


## By user score (released first)

In [17]:
view_group_n(
    games_reviews,
    groupby_fields=["name"],
    value_fields=["user_score", "release_date"],
    agg_funcs=["max", "max"],
    sort_by=["user_score", "release_date"],
    asc=[False, True],
)


,name,user_score,release_date
1,GrimGrimoire,9.7,2007-06-26
2,After Burner Climax,9.7,2010-04-22
3,Metal Torrent,9.7,2010-05-24
4,Z.H.P. Unlosing Ranger vs Darkdeath Evilman,9.7,2010-10-25
5,Ghost Trick: Phantom Detective,9.7,2011-01-11
6,Rochard,9.7,2011-11-15
7,Tengami,9.7,2014-11-13
8,Diaries of a Spaceport Janitor,9.6,2016-09-16
9,Crystar,9.6,2019-08-27
10,Superliminal,9.6,2020-07-07


## By user score (released last)

In [18]:
view_group_n(
    games_reviews,
    groupby_fields=["name"],
    value_fields=["user_score", "release_date"],
    agg_funcs=["max", "max"],
    sort_by=["user_score", "release_date"],
    asc=[False, False],
)


,name,user_score,release_date
1,Tengami,9.7,2014-11-13
2,Rochard,9.7,2011-11-15
3,Ghost Trick: Phantom Detective,9.7,2011-01-11
4,Z.H.P. Unlosing Ranger vs Darkdeath Evilman,9.7,2010-10-25
5,Metal Torrent,9.7,2010-05-24
6,After Burner Climax,9.7,2010-04-22
7,GrimGrimoire,9.7,2007-06-26
8,Superliminal,9.6,2020-07-07
9,Crystar,9.6,2019-08-27
10,Diaries of a Spaceport Janitor,9.6,2016-09-16


## By mean score (released first)

In [19]:
view_group_n(
    games_reviews,
    groupby_fields=["name"],
    value_fields=["mean_score", "release_date"],
    agg_funcs=["max", "max"],
    sort_by=["mean_score", "release_date"],
    asc=[False, True],
)


,name,mean_score,release_date
1,The Legend of Zelda: Ocarina of Time,9.5,1998-11-23
2,Half-Life 2,9.4,2005-11-15
3,Super Mario Galaxy,9.4,2007-11-12
4,The Orange Box,9.4,2007-12-11
5,Super Mario Galaxy 2,9.4,2010-05-23
6,The Witcher 3: Wild Hunt,9.4,2015-05-19
7,Resident Evil 4,9.4,2019-05-21
8,Tekken 3,9.3,1998-04-29
9,Baldur's Gate II: Shadows of Amn,9.3,2000-09-24
10,Metal Gear Solid,9.3,2000-09-24


## By mean score (release last)

In [20]:
view_group_n(
    games_reviews,
    groupby_fields=["name"],
    value_fields=["mean_score", "release_date"],
    agg_funcs=["max", "max"],
    sort_by=["mean_score", "release_date"],
    asc=[False, False],
)


,name,mean_score,release_date
1,The Legend of Zelda: Ocarina of Time,9.5,1998-11-23
2,Resident Evil 4,9.4,2019-05-21
3,The Witcher 3: Wild Hunt,9.4,2015-05-19
4,Super Mario Galaxy 2,9.4,2010-05-23
5,The Orange Box,9.4,2007-12-11
6,Super Mario Galaxy,9.4,2007-11-12
7,Half-Life 2,9.4,2005-11-15
8,God of War,9.3,2018-04-20
9,Super Mario Odyssey,9.3,2017-10-27
10,The Last of Us Remastered,9.3,2014-07-29


## Bottom 10 rated games by meta score, user score and mean score
### *The tiebreaker for games with the same score will be the release date*

## By meta score (released first)

In [25]:
view_group_n(
    games_reviews,
    groupby_fields=["name"],
    value_fields=["meta_score", "release_date"],
    agg_funcs=["min", "min"],
    sort_by=["meta_score", "release_date"],
    asc=[True, True],
)

,name,meta_score,release_date
1,Leisure Suit Larry: Box Office Bust,2.0,2009-03-31
2,Infestation: Survivor Stories (The War Z),2.0,2012-10-15
3,Afro Samurai 2: Revenge of Kuma Volume One,2.1,2015-09-22
4,Drake of the 99 Dragons,2.2,2003-11-03
5,Fast & Furious: Showdown,2.2,2013-05-21
6,Charlie's Angels,2.3,2003-07-09
7,Homie Rollerz,2.3,2008-03-05
8,FlatOut 3: Chaos & Destruction,2.3,2011-12-13
9,Fighter Within,2.3,2013-11-19
10,Rambo: The Video Game,2.3,2014-02-21


## By meta score (released last)

In [26]:
view_group_n(
    games_reviews,
    groupby_fields=["name"],
    value_fields=["meta_score", "release_date"],
    agg_funcs=["min", "min"],
    sort_by=["meta_score", "release_date"],
    asc=[True, False],
)

,name,meta_score,release_date
1,Infestation: Survivor Stories (The War Z),2.0,2012-10-15
2,Leisure Suit Larry: Box Office Bust,2.0,2009-03-31
3,Afro Samurai 2: Revenge of Kuma Volume One,2.1,2015-09-22
4,Fast & Furious: Showdown,2.2,2013-05-21
5,Drake of the 99 Dragons,2.2,2003-11-03
6,Rambo: The Video Game,2.3,2014-02-21
7,Fighter Within,2.3,2013-11-19
8,FlatOut 3: Chaos & Destruction,2.3,2011-12-13
9,Homie Rollerz,2.3,2008-03-05
10,Charlie's Angels,2.3,2003-07-09


## By user score (released first)

In [28]:
view_group_n(
    games_reviews[games_reviews.user_score != 0],
    groupby_fields=["name"],
    value_fields=["user_score", "release_date"],
    agg_funcs=["min", "min"],
    sort_by=["user_score", "release_date"],
    asc=[True, True],
)

,name,user_score,release_date
1,Madden NFL 21,0.2,2020-08-25
2,Madden NFL 22,0.5,2021-08-17
3,Warcraft III: Reforged,0.6,2020-01-28
4,FIFA 20: Legacy Edition,0.7,2019-09-27
5,NBA 2K21,0.8,2020-09-04
6,FIFA 21,0.8,2020-10-06
7,Tom Clancy's The Division 2: Warlords of New York,0.9,2020-03-03
8,When Ski Lifts Go Wrong,1.0,2019-01-23
9,The Sims 4: Star Wars - Journey to Batuu,1.0,2020-09-08
10,WWE 2K20,1.1,2019-10-22


## By user score (released last)

In [29]:
view_group_n(
    games_reviews[games_reviews.user_score != 0],
    groupby_fields=["name"],
    value_fields=["user_score", "release_date"],
    agg_funcs=["min", "min"],
    sort_by=["user_score", "release_date"],
    asc=[True, False],
)

,name,user_score,release_date
1,Madden NFL 21,0.2,2020-08-25
2,Madden NFL 22,0.5,2021-08-17
3,Warcraft III: Reforged,0.6,2020-01-28
4,FIFA 20: Legacy Edition,0.7,2019-09-27
5,FIFA 21,0.8,2020-10-06
6,NBA 2K21,0.8,2020-09-04
7,Tom Clancy's The Division 2: Warlords of New York,0.9,2020-03-03
8,The Sims 4: Star Wars - Journey to Batuu,1.0,2020-09-08
9,When Ski Lifts Go Wrong,1.0,2019-01-23
10,WWE 2K20,1.1,2019-10-22


## By mean score (released first)

In [30]:
view_group_n(
    games_reviews,
    groupby_fields=["name"],
    value_fields=["mean_score", "release_date"],
    agg_funcs=["min", "min"],
    sort_by=["mean_score", "release_date"],
    asc=[True, True],
)

,name,mean_score,release_date
1,Fast & Furious: Showdown,1.8,2013-05-21
2,Infestation: Survivor Stories (The War Z),1.9,2012-10-15
3,Drake of the 99 Dragons,2.0,2003-11-03
4,Leisure Suit Larry: Box Office Bust,2.0,2009-03-31
5,R.I.P.D. The Game,2.0,2013-07-16
6,Rambo: The Video Game,2.0,2014-02-21
7,Dragon Ball: Evolution,2.1,2009-04-08
8,NBA Unrivaled,2.1,2009-11-11
9,Thundercats,2.2,2012-10-30
10,Ghostbusters,2.2,2016-07-12


## By mean score (released last)

In [31]:
view_group_n(
    games_reviews,
    groupby_fields=["name"],
    value_fields=["mean_score", "release_date"],
    agg_funcs=["min", "min"],
    sort_by=["mean_score", "release_date"],
    asc=[True, False],
)

,name,mean_score,release_date
1,Fast & Furious: Showdown,1.8,2013-05-21
2,Infestation: Survivor Stories (The War Z),1.9,2012-10-15
3,Rambo: The Video Game,2.0,2014-02-21
4,R.I.P.D. The Game,2.0,2013-07-16
5,Leisure Suit Larry: Box Office Bust,2.0,2009-03-31
6,Drake of the 99 Dragons,2.0,2003-11-03
7,NBA Unrivaled,2.1,2009-11-11
8,Dragon Ball: Evolution,2.1,2009-04-08
9,Wild West Online,2.2,2018-05-10
10,Ghostbusters,2.2,2016-07-12
